## **1. Import the necessary libraries** ---

In [1]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

print("Versions of key libraries")
print("---")
print("tensorflow: ", tf.__version__)
print("numpy:      ", np.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)

Versions of key libraries
---
tensorflow:  2.3.0
numpy:       1.18.5
matplotlib:  3.4.2
sklearn:     0.24.2


## **2.Create a function to plot image without axis** ---

In [2]:
def implt(img):
    plt.figure()
    plt.imshow(img)
    plt.axis('off')

print(implt)

<function implt at 0x0000024A79E90678>


## **3. Set matplotlib to have seaborn plot style**

In [3]:
plt.style.use('seaborn')                   # if want to use the default style, set 'classic'
plt.rcParams['ytick.right']     = True
plt.rcParams['ytick.labelright']= True
plt.rcParams['ytick.left']      = False
plt.rcParams['ytick.labelleft'] = False
plt.rcParams['figure.figsize']  = [7,7]   # Set the figure size to be 7 inch for (width,height)

print("Matplotlib setup completes.")

Matplotlib setup completes.


## **4. Prepare data for training and testing**
---
* Step 1: Load the images
* Step 2: Check the shape and type of the data
* Step 3: Convert the data into float32 and rescale the values from the range of 0\~255 into 0\~1
* Step 4: Retrieve the row size and the column size of each image
* Step 5: Perform one-hot enconding on the labels
* Step 6: Retrieve the number of classes in this problem

In [4]:
from imutils import paths
import os, sys
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def collect_images_and_labels(path_to_images):
    # in seguito, usare https://keras.io/api/preprocessing/image/
    """
        :param path_to_images should be the root folder, in which there is a folder for each label, and the folder's name is
        the label itself
        :return: a list with images and a list with labels
        """
    data = []
    labels = []

    for img_path in list(paths.list_images(path_to_images)):
        # extract the class label from the filename
        label = img_path.split(os.path.sep)[-2]

        # load the input image as (224x224) and preprocess it
        image = load_img(img_path, target_size=(128, 128))
        image = img_to_array(image)
        #image = preprocess_input(image)

        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

    return data, labels

def preprocess_labels(labels):
    """
    :param labels: list of string
    :return: np array of 0/1
    """
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = to_categorical(labels)
    return labels

def tts_split(data, labels):
    (x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.20, stratify=labels,
                                                          random_state=42)
    (x_train, x_test, y_train, y_test) = (np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test))
    return x_train, x_test, y_train, y_test

PATH_TO_IMAGE = "../data"
MODELS_PATH = "../models"

# Step 1

data, labels = collect_images_and_labels(PATH_TO_IMAGE)
#labels = preprocess_labels(labels)
trDat, tsDat, trLbl, tsLbl = tts_split(data, labels)


# Step 2
print("The shape of trDat is", trDat.shape, "and the type of trDat is", trDat.dtype)
print("The shape of tsDat is", tsDat.shape, "and the type of tsDat is", tsDat.dtype)
print("")
print("The shape of trLbl is", trLbl.shape, "and the type of trLbl is", trLbl.dtype)
print("The shape of tsLbl is", tsLbl.shape, "and the type of tsLbl is", tsLbl.dtype)


# Step 3
trDat           = trDat.astype('float32')/255
tsDat           = tsDat.astype('float32')/255

# Step 4
imgrows         = trDat.shape[1]
imgclms         = trDat.shape[2]
channel         = trDat.shape[3]

# Step 5
trLbl           = preprocess_labels(trLbl)
tsLbl           = preprocess_labels(tsLbl)

# Step 6
num_classes     = tsLbl.shape[1]           

C:\Users\HONG0\.conda\envs\detective_mask\lib\site-packages\PIL\Image.py:974: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


The shape of trDat is (6042, 128, 128, 3) and the type of trDat is float32
The shape of tsDat is (1511, 128, 128, 3) and the type of tsDat is float32

The shape of trLbl is (6042,) and the type of trLbl is <U12
The shape of tsLbl is (1511,) and the type of tsLbl is <U12


## **5. Define the wBNRg model (to be completed)**
___
* Step 1: Setup the optimizer to be used for training
* Step 2: Set a name for the coming model (required for saving)
* Step 3: Define the convolutional neural network model (to be completed)
* Step 4: Create models for training and testing
* Step 5: Display the summary of the model of interest 

In [5]:
# Step 1
optmz       = optimizers.RMSprop(lr=0.0001)
# Step 2
modelname   = 'DetecitveMask'                                                           

# Step 3                                                                               
def createModel():
    
    ipt = Input(shape=(imgrows, imgclms, channel))
    x = Conv2D(128,(3,3),padding='same')(ipt)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(64,(3,3),padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(32,(3,3),padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=ipt, outputs=x)
#    if (tf.test.is_built_with_cuda()):
#        from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
#        model = multi_gpu_model(model, gpus=0)
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

# Step 4                                                                                
model       = createModel() # This is meant for training
modelGo     = createModel() # This is used for final testing

# Step 5
model.summary()                                                                 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 128)     3584      
_________________________________________________________________
activation (Activation)      (None, 128, 128, 128)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        73792     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)       

## **6. Create the checkpoints to be applied during training**
---
* Step 1: Create a checkpoint to save the model from an epoch when validation accuracy is the highest
* Step 2: Create a checkpoint to save the training loss, training accuracy, validation loss and validation accuracy of each epoch into a csv file
* Step 3: Put the two checkpoint objects into a list

In [6]:
# Step 1
folderpath      = MODELS_PATH + '/'
filepath        = folderpath + modelname + ".hdf5"
checkpoint      = ModelCheckpoint(filepath, 
                                  monitor='val_accuracy', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')

# Step 2
csv_logger      = CSVLogger(folderpath+modelname +'.csv')

# Step 3
callbacks_list  = [checkpoint,csv_logger]                                       

print("Callbacks created:")
print(callbacks_list[0])
print(callbacks_list[1])
print('')
print("Path to model:", filepath)
print("Path to log:  ", folderpath+modelname+'.csv')

Callbacks created:

Path to model: ../models/DetecitveMask.hdf5
Path to log:   ../models/DetecitveMask.csv


## **8. Train the deep learning model**

In [7]:

model.fit(trDat,                    # Training data
          trLbl,                            # Training label
          validation_data=(tsDat, tsLbl),   # Validation data and label
          epochs=100,                       # The amount of epochs to be trained
          batch_size=64,                   
          shuffle=True,                     # To shuffle the training data
          callbacks=callbacks_list)         # Callbacks to execute the checkpoints

Epoch 1/100
95/95 [==============================] - 130s 1s/step - loss: 0.5309 - accuracy: 0.7315 - val_loss: 0.3135 - val_accuracy: 0.8709
Epoch 2/100
95/95 [==============================] - 153s 2s/step - loss: 0.3049 - accuracy: 0.8762 - val_loss: 0.2320 - val_accuracy: 0.9219
Epoch 3/100
95/95 [==============================] - 168s 2s/step - loss: 0.2401 - accuracy: 0.9057 - val_loss: 0.4426 - val_accuracy: 0.8299
Epoch 4/100
95/95 [==============================] - 171s 2s/step - loss: 0.1923 - accuracy: 0.9308 - val_loss: 0.2435 - val_accuracy: 0.9001
Epoch 5/100
95/95 [==============================] - 174s 2s/step - loss: 0.1535 - accuracy: 0.9451 - val_loss: 0.2322 - val_accuracy: 0.9153
Epoch 6/100
95/95 [==============================] - 174s 2s/step - loss: 0.1293 - accuracy: 0.9553 - val_loss: 0.1420 - val_accuracy: 0.9457
Epoch 7/100
95/95 [==============================] - 176s 2s/step - loss: 0.1128 - accuracy: 0.9608 - val_loss: 0.1487 - val_accuracy: 0.9563
Epoch 

95/95 [==============================] - 179s 2s/step - loss: 0.0221 - accuracy: 0.9967 - val_loss: 1.2499 - val_accuracy: 0.9517
Epoch 59/100
95/95 [==============================] - 179s 2s/step - loss: 0.0212 - accuracy: 0.9974 - val_loss: 1.0146 - val_accuracy: 0.9391
Epoch 60/100
95/95 [==============================] - 178s 2s/step - loss: 0.0220 - accuracy: 0.9970 - val_loss: 1.1733 - val_accuracy: 0.9358
Epoch 61/100
95/95 [==============================] - 180s 2s/step - loss: 0.0104 - accuracy: 0.9983 - val_loss: 1.0456 - val_accuracy: 0.9497
Epoch 62/100
95/95 [==============================] - 180s 2s/step - loss: 0.0155 - accuracy: 0.9964 - val_loss: 1.3798 - val_accuracy: 0.9365
Epoch 63/100
95/95 [==============================] - 180s 2s/step - loss: 0.0113 - accuracy: 0.9982 - val_loss: 1.4355 - val_accuracy: 0.9451
Epoch 64/100
95/95 [==============================] - 180s 2s/step - loss: 0.0159 - accuracy: 0.9980 - val_loss: 1.5257 - val_accuracy: 0.9504
Epoch 65/100

In [12]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [13]:
get_available_gpus()

['/device:GPU:0']

In [10]:
tf.test.is_built_with_cuda()

True

In [11]:
device_lib.list_local_devices()

NameError: name 'device_lib' is not defined

In [14]:
tf.test.gpu_device_name()

''

In [1]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']
